In [5]:
%pip install numpy pandas duckdb

%pip install duckdb


import duckdb

con = duckdb.connect(database=":memory:")

con.execute(
    "CREATE TABLE raw_data AS SELECT * FROM read_csv_auto('/Users/sripaachu/Downloads/data.csv', encoding='latin-1')"
)

con.sql("SELECT COUNT(*) AS row_count FROM raw_data").df()


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.3 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 10.8 MB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 348 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 20.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


,row_count
0,541909


In [7]:
# Inspect table structure (column names and inferred types)
con.sql("DESCRIBE raw_data").df()

# Preview first 10 rows
con.sql("SELECT * FROM raw_data LIMIT 10").df()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047,United Kingdom


In [8]:
# Check missing values (total NULLs for key columns)
con.sql("""
    SELECT
        SUM(CASE WHEN InvoiceNo   IS NULL THEN 1 ELSE 0 END) AS null_invoiceno,
        SUM(CASE WHEN StockCode   IS NULL THEN 1 ELSE 0 END) AS null_stockcode,
        SUM(CASE WHEN Description IS NULL THEN 1 ELSE 0 END) AS null_description,
        SUM(CASE WHEN Quantity    IS NULL THEN 1 ELSE 0 END) AS null_quantity,
        SUM(CASE WHEN InvoiceDate IS NULL THEN 1 ELSE 0 END) AS null_invoicedate,
        SUM(CASE WHEN UnitPrice   IS NULL THEN 1 ELSE 0 END) AS null_unitprice,
        SUM(CASE WHEN CustomerID  IS NULL THEN 1 ELSE 0 END) AS null_customerid,
        SUM(CASE WHEN Country     IS NULL THEN 1 ELSE 0 END) AS null_country
    FROM raw_data
""").df()

# Check invalid numeric values (Quantity <= 0)
con.sql("""
    SELECT COUNT(*) AS rows_quantity_le_0
    FROM raw_data
    WHERE Quantity <= 0
""").df()

# Check invalid numeric values (UnitPrice <= 0)
con.sql("""
    SELECT COUNT(*) AS rows_unitprice_le_0
    FROM raw_data
    WHERE UnitPrice <= 0
""").df()

# Check cancelled transactions (InvoiceNo starts with 'C')
con.sql("""
    SELECT COUNT(*) AS cancelled_rows
    FROM raw_data
    WHERE InvoiceNo LIKE 'C%'
""").df()

,cancelled_rows
0,9288


In [9]:
# Overlap: negative/zero quantities that are cancellations
con.sql("""
    SELECT COUNT(*) AS rows_qty_le_0_and_cancelled
    FROM raw_data
    WHERE Quantity <= 0
      AND InvoiceNo LIKE 'C%'
""").df()

# Overlap: negative/zero quantities that are not cancellations
con.sql("""
    SELECT COUNT(*) AS rows_qty_le_0_and_not_cancelled
    FROM raw_data
    WHERE Quantity <= 0
      AND InvoiceNo NOT LIKE 'C%'
""").df()

# Flag: non-positive prices with positive quantities
con.sql("""
    SELECT COUNT(*) AS rows_unitprice_le_0_and_qty_gt_0
    FROM raw_data
    WHERE UnitPrice <= 0
      AND Quantity > 0
""").df()

,rows_unitprice_le_0_and_qty_gt_0
0,1181


In [10]:
# Create a clean view for analysis-ready sales records
con.sql("""
    CREATE OR REPLACE VIEW valid_sales AS
    SELECT *
    FROM raw_data
    WHERE InvoiceNo NOT LIKE 'C%'
      AND Quantity > 0
      AND UnitPrice > 0
""")

# Confirm how many rows remain after cleaning
con.sql("""
    SELECT COUNT(*) AS valid_sales_row_count
    FROM valid_sales
""").df()

# Preview a few cleaned rows
con.sql("""
    SELECT *
    FROM valid_sales
    LIMIT 5
""").df()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [ ]:
# Total revenue (Quantity * UnitPrice)
con.sql("""
    SELECT
        SUM(Quantity * UnitPrice) AS total_revenue
    FROM valid_sales
""").df()

# Date range (parse InvoiceDate, then take min/max)
con.sql("""
    SELECT
        MIN(invoice_ts) AS min_invoice_datetime,
        MAX(invoice_ts) AS max_invoice_datetime
    FROM (
        SELECT try_strptime(InvoiceDate, '%m/%d/%Y %H:%M') AS invoice_ts
        FROM valid_sales
    )
""").df()

# Average order value (AOV): revenue per InvoiceNo, then average across invoices
con.sql("""
    WITH invoice_revenue AS (
        SELECT
            InvoiceNo,
            SUM(Quantity * UnitPrice) AS revenue
        FROM valid_sales
        GROUP BY InvoiceNo
    )
    SELECT
        AVG(revenue) AS avg_order_value,
        COUNT(*) AS invoice_count
    FROM invoice_revenue
""").df()

In [ ]:
# Monthly revenue trend (group by year-month)
con.sql("""
    WITH base AS (
        SELECT
            InvoiceNo,
            Quantity,
            UnitPrice,
            date_trunc('month', try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS month_start
        FROM valid_sales
    )
    SELECT
        month_start,
        SUM(Quantity * UnitPrice) AS monthly_revenue
    FROM base
    GROUP BY month_start
    ORDER BY month_start
""").df()

# Monthly order volume trend (distinct invoices per month)
con.sql("""
    WITH base AS (
        SELECT
            InvoiceNo,
            date_trunc('month', try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS month_start
        FROM valid_sales
    )
    SELECT
        month_start,
        COUNT(DISTINCT InvoiceNo) AS monthly_order_count
    FROM base
    GROUP BY month_start
    ORDER BY month_start
""").df()

In [11]:
# Final confirmation: number of clean sales rows
con.sql("SELECT COUNT(*) AS valid_sales_rows FROM valid_sales").df()


,valid_sales_rows
0,530104


In [12]:
# Calculate total revenue from valid sales
con.sql("""
SELECT
    SUM(Quantity * UnitPrice) AS total_revenue
FROM valid_sales
""").df()


# Check the date range of the dataset
con.sql("""
SELECT
    MIN(try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS min_invoice_datetime,
    MAX(try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS max_invoice_datetime
FROM valid_sales
""").df()


# Calculate Average Order Value (AOV)
# Step 1: revenue per invoice
# Step 2: average across invoices
con.sql("""
WITH invoice_revenue AS (
    SELECT
        InvoiceNo,
        SUM(Quantity * UnitPrice) AS revenue
    FROM valid_sales
    GROUP BY InvoiceNo
)
SELECT
    AVG(revenue) AS avg_order_value,
    COUNT(*) AS invoice_count
FROM invoice_revenue
""").df()


,avg_order_value,invoice_count
0,534.403033,19960


In [13]:
# Calculate total revenue from valid sales
con.sql("""
SELECT
    SUM(Quantity * UnitPrice) AS total_revenue
FROM valid_sales
""").df()


,total_revenue
0,1.066668e+07


In [14]:
# Check the date range of the dataset
con.sql("""
SELECT
    MIN(try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS min_invoice_datetime,
    MAX(try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS max_invoice_datetime
FROM valid_sales
""").df()


,min_invoice_datetime,max_invoice_datetime
0,2010-12-01 08:26:00,2011-12-09 12:50:00


In [15]:
# Monthly revenue trend (group by year-month)
con.sql("""
    WITH base AS (
        SELECT
            InvoiceNo,
            Quantity,
            UnitPrice,
            date_trunc('month', try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS month_start
        FROM valid_sales
    )
    SELECT
        month_start,
        SUM(Quantity * UnitPrice) AS monthly_revenue
    FROM base
    GROUP BY month_start
    ORDER BY month_start
""").df()

# Monthly order volume trend (distinct invoices per month)
con.sql("""
    WITH base AS (
        SELECT
            InvoiceNo,
            date_trunc('month', try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS month_start
        FROM valid_sales
    )
    SELECT
        month_start,
        COUNT(DISTINCT InvoiceNo) AS monthly_order_count
    FROM base
    GROUP BY month_start
    ORDER BY month_start
""").df()

,month_start,monthly_order_count
0,2010-12-01,1559
1,2011-01-01,1086
2,2011-02-01,1100
3,2011-03-01,1454
4,2011-04-01,1246
5,2011-05-01,1681
6,2011-06-01,1533
7,2011-07-01,1475
8,2011-08-01,1361
9,2011-09-01,1837


In [16]:
# Monthly performance table (revenue + distinct orders per month)
con.sql("""
    WITH base AS (
        SELECT
            InvoiceNo,
            Quantity,
            UnitPrice,
            date_trunc('month', try_strptime(InvoiceDate, '%m/%d/%Y %H:%M')) AS month_start
        FROM valid_sales
    )
    SELECT
        month_start,
        SUM(Quantity * UnitPrice) AS monthly_revenue,
        COUNT(DISTINCT InvoiceNo) AS monthly_order_count
    FROM base
    GROUP BY month_start
    ORDER BY month_start
""").df()

,month_start,monthly_revenue,monthly_order_count
0,2010-12-01,823746.140,1559
1,2011-01-01,691364.560,1086
2,2011-02-01,523631.890,1100
3,2011-03-01,717639.360,1454
4,2011-04-01,537808.621,1246
5,2011-05-01,770536.020,1681
6,2011-06-01,761739.900,1533
7,2011-07-01,719221.191,1475
8,2011-08-01,759138.380,1361
9,2011-09-01,1058590.172,1837


In [17]:
# Top 10 products by total revenue (group by product identifiers)
con.sql("""
    SELECT
        StockCode,
        Description,
        SUM(Quantity * UnitPrice) AS total_revenue
    FROM valid_sales
    GROUP BY
        StockCode,
        Description
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

,StockCode,Description,total_revenue
0,DOT,DOTCOM POSTAGE,206248.77
1,22423,REGENCY CAKESTAND 3 TIER,174484.74
2,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60
3,85123A,WHITE HANGING HEART T-LIGHT HOLDER,104340.29
4,47566,PARTY BUNTING,99504.33
5,85099B,JUMBO BAG RED RETROSPOT,94340.05
6,23166,MEDIUM CERAMIC TOP STORAGE JAR,81700.92
7,M,Manual,78110.27
8,POST,POSTAGE,78101.88
9,23084,RABBIT NIGHT LIGHT,66964.99


In [ ]:
# Revenue concentration across products (rank + cumulative share)
con.sql("""
    WITH product_revenue AS (
        SELECT
            StockCode,
            Description,
            SUM(Quantity * UnitPrice) AS product_revenue
        FROM valid_sales
        GROUP BY
            StockCode,
            Description
    ), ranked AS (
        SELECT
            StockCode,
            Description,
            product_revenue,
            ROW_NUMBER() OVER (ORDER BY product_revenue DESC) AS revenue_rank,
            SUM(product_revenue) OVER (
                ORDER BY product_revenue DESC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS cumulative_revenue,
            SUM(product_revenue) OVER () AS total_revenue
        FROM product_revenue
    )
    SELECT
        Description,
        product_revenue,
        cumulative_revenue / NULLIF(total_revenue, 0) AS cumulative_revenue_pct
    FROM ranked
    ORDER BY revenue_rank
    LIMIT 20
""").df()

Top revenue drivers include non-product charges such as postage, which should be excluded when analysing product performance.

Revenue concentration is driven by a small set of gift-oriented decorative products, which aligns with strong Q4 seasonality observed in the time-series analysis.

In [18]:
# Revenue concentration across products (rank + cumulative share)
con.sql("""
    WITH product_revenue AS (
        SELECT
            StockCode,
            Description,
            SUM(Quantity * UnitPrice) AS product_revenue
        FROM valid_sales
        GROUP BY
            StockCode,
            Description
    ), ranked AS (
        SELECT
            StockCode,
            Description,
            product_revenue,
            ROW_NUMBER() OVER (ORDER BY product_revenue DESC) AS revenue_rank,
            SUM(product_revenue) OVER (
                ORDER BY product_revenue DESC
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) AS cumulative_revenue,
            SUM(product_revenue) OVER () AS total_revenue
        FROM product_revenue
    )
    SELECT
        Description,
        product_revenue,
        cumulative_revenue / NULLIF(total_revenue, 0) AS cumulative_revenue_pct
    FROM ranked
    ORDER BY revenue_rank
    LIMIT 20
""").df()

,Description,product_revenue,cumulative_revenue_pct
0,DOTCOM POSTAGE,206248.77,0.019336
1,REGENCY CAKESTAND 3 TIER,174484.74,0.035694
2,"PAPER CRAFT , LITTLE BIRDIE",168469.60,0.051488
3,WHITE HANGING HEART T-LIGHT HOLDER,104340.29,0.061270
4,PARTY BUNTING,99504.33,0.070598
5,JUMBO BAG RED RETROSPOT,94340.05,0.079442
6,MEDIUM CERAMIC TOP STORAGE JAR,81700.92,0.087102
7,Manual,78110.27,0.094425
8,POSTAGE,78101.88,0.101747
9,RABBIT NIGHT LIGHT,66964.99,0.108025


In [ ]:
# Customer repeat behaviour (bucket customers by number of orders)
con.sql("""
    WITH orders_per_customer AS (
        SELECT
            CustomerID,
            COUNT(DISTINCT InvoiceNo) AS order_count
        FROM valid_sales
        WHERE CustomerID IS NOT NULL
        GROUP BY CustomerID
    ), bucketed AS (
        SELECT
            CASE
                WHEN order_count = 1 THEN '1 order'
                WHEN order_count BETWEEN 2 AND 5 THEN '2–5 orders'
                WHEN order_count BETWEEN 6 AND 10 THEN '6–10 orders'
                ELSE '>10 orders'
            END AS order_bucket
        FROM orders_per_customer
    ), bucket_counts AS (
        SELECT
            order_bucket,
            COUNT(*) AS customer_count
        FROM bucketed
        GROUP BY order_bucket
    )
    SELECT
        order_bucket,
        customer_count,
        customer_count * 1.0 / SUM(customer_count) OVER () AS pct_of_customers
    FROM bucket_counts
    ORDER BY
        CASE order_bucket
            WHEN '1 order' THEN 1
            WHEN '2–5 orders' THEN 2
            WHEN '6–10 orders' THEN 3
            ELSE 4
        END
""").df()

Revenue is not overly concentrated in a small number of products, indicating a diversified product portfolio.

Operational charges appear among top revenue contributors and should be excluded when analysing pure product performance.


In [ ]:
# Customer repeat behaviour (bucket customers by number of orders)
con.sql("""
    WITH orders_per_customer AS (
        SELECT
            CustomerID,
            COUNT(DISTINCT InvoiceNo) AS order_count
        FROM valid_sales
        WHERE CustomerID IS NOT NULL
        GROUP BY CustomerID
    ), bucketed AS (
        SELECT
            CASE
                WHEN order_count = 1 THEN '1 order'
                WHEN order_count BETWEEN 2 AND 5 THEN '2–5 orders'
                WHEN order_count BETWEEN 6 AND 10 THEN '6–10 orders'
                ELSE '>10 orders'
            END AS order_bucket
        FROM orders_per_customer
    ), bucket_counts AS (
        SELECT
            order_bucket,
            COUNT(*) AS customer_count
        FROM bucketed
        GROUP BY order_bucket
    )
    SELECT
        order_bucket,
        customer_count,
        customer_count * 1.0 / SUM(customer_count) OVER () AS pct_of_customers
    FROM bucket_counts
    ORDER BY
        CASE order_bucket
            WHEN '1 order' THEN 1
            WHEN '2–5 orders' THEN 2
            WHEN '6–10 orders' THEN 3
            ELSE 4
        END
""").df()